In [1]:
import hopsworks
from hsfs.client.exceptions import RestAPIError

# Establish connection to Hopsworks
project = hopsworks.login(
    api_key_value="dYTVrTVvbj6Qw82i.YGKHdS9snQYFgOADJIvLdvZ2n2S5BxIAOtvPUEmAyd56bvaG6xhhGyNM3nYbexaP"
)

# Access the Feature Store
fs = project.get_feature_store()

# List of feature groups and their versions
feature_groups = [
    {"name": "city_weather", "version": 1},
    {"name": "drivers_table", "version": 1},
    {"name": "routes_table", "version": 1},
    {"name": "routes_weather", "version": 1},
    {"name": "traffic_table", "version": 1},  # Corrected typo
    {"name": "truck_schedule_table", "version": 1},
    {"name": "trucks_table", "version": 1},
]
# Initialize an empty dictionary to hold DataFrames for each feature group
feature_group_data = {}

# Loop over the list of feature groups and download them
for fg in feature_groups:
    try:
        # Retrieve the feature group by its name and version
        feature_group = fs.get_feature_group(fg['name'], version=fg['version'])
        df = feature_group.read()  # Read the feature group as a DataFrame
        
        # Save the DataFrame in the dictionary with the feature group's name as the key
        feature_group_data[fg['name']] = df

        print(f"Downloaded feature group: {fg['name']} (version {fg['version']})")
        print(df.head())  # Optionally display the first few rows of the DataFrame

    except RestAPIError as e:
        print(f"Error downloading feature group: {fg['name']} (version {fg['version']})")
        print(e)

# After the loop, you will have each feature group stored in the `feature_group_data` dictionary
# Access each DataFrame like this: feature_group_data['city_weather'], etc.



Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1044630
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.24s) 
Downloaded feature group: city_weather (version 1)
      city_id  temp  wind_speed           description  precip  humidity  \
0  C-731988ba    52           1  Patchy rain possible     0.0        80   
1  C-841ebdcb    66           6         Partly cloudy     0.0        84   
2  C-ef47bdcd    10           6                 Sunny     0.0        57   
3  C-ff8c0c3c    34           8              Overcast     0.0       100   
4  C-3dbd8b2e    41          11         Partly cloudy     0.0        73   

   visibility  pressure     id                       event_time  \
0           6      1015   8790 2024-10-10 20:13:27.088206+00:00   
1           6      1020  52029 2024-10-10 20:13:27.088206+00:00   
2    

In [2]:
feature_group_data.keys()

dict_keys(['city_weather', 'drivers_table', 'routes_table', 'routes_weather', 'traffic_table', 'truck_schedule_table', 'trucks_table'])

In [3]:
city_weather = feature_group_data['city_weather']
drivers_table = feature_group_data['drivers_table']
routes_table = feature_group_data['routes_table']
routes_weather = feature_group_data['routes_weather']
traffic_table = feature_group_data['traffic_table'] 
truck_schedule_table = feature_group_data['truck_schedule_table']
trucks_table = feature_group_data['trucks_table']


city_weather.head()

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,id,event_time,datetime
0,C-731988ba,52,1,Patchy rain possible,0.0,80,6,1015,8790,2024-10-10 20:13:27.088206+00:00,2019-02-14 05:00:00+00:00
1,C-841ebdcb,66,6,Partly cloudy,0.0,84,6,1020,52029,2024-10-10 20:13:27.088206+00:00,2019-01-07 20:00:00+00:00
2,C-ef47bdcd,10,6,Sunny,0.0,57,6,1036,13063,2024-10-10 20:13:27.088206+00:00,2019-02-08 06:00:00+00:00
3,C-ff8c0c3c,34,8,Overcast,0.0,100,6,1024,16910,2024-10-10 20:13:27.088206+00:00,2019-01-15 13:00:00+00:00
4,C-3dbd8b2e,41,11,Partly cloudy,0.0,73,6,1021,18928,2024-10-10 20:13:27.088206+00:00,2019-01-07 15:00:00+00:00


In [4]:
routes_weather.head()

,route_id,date,temp,wind_speed,description,precip,humidity,visibility,pressure,id,event_time
0,R-566db3fc,2019-01-21 12:00:00+00:00,28,16,Moderate or heavy snow showers,0.0,73,4,1015,245303,2024-10-10 20:13:27.063199+00:00
1,R-05a6b7e1,2019-02-13 18:00:00+00:00,16,4,Clear,0.0,63,6,1021,397270,2024-10-10 20:13:27.063199+00:00
2,R-79047854,2019-01-31 00:00:00+00:00,55,7,Sunny,0.0,57,6,1022,98159,2024-10-10 20:13:27.063199+00:00
3,R-e0263659,2019-01-22 00:00:00+00:00,75,4,Cloudy,0.0,66,6,1016,30437,2024-10-10 20:13:27.063199+00:00
4,R-dcbddbb7,2019-02-01 12:00:00+00:00,61,3,Clear,0.0,89,6,1011,259995,2024-10-10 20:13:27.063199+00:00


In [5]:
def assign_feature_group_data(feature_group_data, table_names):
    # Print available keys in feature_group_data for debugging
    print("Available keys in feature_group_data:", list(feature_group_data.keys()))
    
    # Dictionary to store dynamically assigned DataFrames
    assigned_dataframes = {}

    # Dynamically assign the DataFrame from feature_group_data to the dictionary
    for table_name in table_names:
        if table_name in feature_group_data:
            assigned_dataframes[table_name] = feature_group_data[table_name]
            print(f"Assigned {table_name} to assigned_dataframes.")
        else:
            print(f"KeyError: '{table_name}' not found in feature_group_data.")
    
    return assigned_dataframes

# List of table names matching the keys exactly as they appear in 'feature_group_data'
table_names = ['city_weather', 'routes_weather', 'traffic_table', 'truck_schedule_table', 'trucks_table']

# Call the function to assign feature group data to variables
assigned_dataframes = assign_feature_group_data(feature_group_data, table_names)

# Now you can access the DataFrames from the dictionary
print(assigned_dataframes['truck_schedule_table'].head())


Available keys in feature_group_data: ['city_weather', 'drivers_table', 'routes_table', 'routes_weather', 'traffic_table', 'truck_schedule_table', 'trucks_table']
Assigned city_weather to assigned_dataframes.
Assigned routes_weather to assigned_dataframes.
Assigned traffic_table to assigned_dataframes.
Assigned truck_schedule_table to assigned_dataframes.
Assigned trucks_table to assigned_dataframes.
   truck_id    route_id            departure_date         estimated_arrival  \
0  23244180  R-b1e8e9c8 2019-02-06 07:00:00+00:00 2019-02-07 04:09:00+00:00   
1  10234289  R-d053f5bc 2019-01-22 07:00:00+00:00 2019-01-22 22:51:00+00:00   
2  26739096  R-bf6ee666 2019-01-07 07:00:00+00:00 2019-01-07 15:54:36+00:00   
3  20067958  R-abc9e78b 2019-01-04 07:00:00+00:00 2019-01-05 05:28:12+00:00   
4  23639934  R-7dac228a 2019-01-31 07:00:00+00:00 2019-01-31 13:40:12+00:00   

   delay    id                       event_time  
0      1  7633 2024-10-10 20:13:27.142196+00:00  
1      0  4651 2024-1

In [6]:

dfs = {
    'city_weather': city_weather,
    'drivers_table': drivers_table,
    'routes_table': routes_table,
    'routes_weather': routes_weather,
    'traffic_table': traffic_table,
    'truck_schedule_table': truck_schedule_table,
    'trucks_table': trucks_table
}


def drop_columns_from_tables(dfs, columns_to_drop):
    """
    Drops specified columns from all DataFrames in the provided dictionary.
    
    Parameters:
    - dfs (dict): A dictionary where the keys are DataFrame names and the values are pandas DataFrames.
    - columns_to_drop (list): A list of column names to be dropped from each DataFrame.
    
    Returns:
    - dict: A dictionary of DataFrames with the specified columns removed.
    """
    
    # Loop through each DataFrame in the dictionary
    for table_name, df in dfs.items():
        # Check if the columns exist in the DataFrame before dropping
        cols_to_drop = [col for col in columns_to_drop if col in df.columns]
        
        # If there are columns to drop, drop them
        if cols_to_drop:
            dfs[table_name] = df.drop(columns=cols_to_drop, inplace=True)
            print(f"Dropped columns {cols_to_drop} from {table_name}.")
        else:
            print(f"No matching columns to drop in {table_name}.")
    
    return dfs

# Example usage:
columns_to_drop = ['id', 'event_time']  # Example columns that might be present in multiple tables
feature_group_data_cleaned = drop_columns_from_tables(feature_group_data, columns_to_drop)




Dropped columns ['id', 'event_time'] from city_weather.
Dropped columns ['id', 'event_time'] from drivers_table.
Dropped columns ['id', 'event_time'] from routes_table.
Dropped columns ['id', 'event_time'] from routes_weather.
Dropped columns ['id', 'event_time'] from traffic_table.
Dropped columns ['id', 'event_time'] from truck_schedule_table.
Dropped columns ['id', 'event_time'] from trucks_table.


In [7]:
def drop_duplicates_from_dfs(dfs, columns_to_check):   
    for df_name, df in dfs.items():
        if df_name in columns_to_check:
            subset_columns = columns_to_check[df_name]
            
            # Check if all subset columns exist in the DataFrame
            missing_columns = [col for col in subset_columns if col not in df.columns]
            if missing_columns:
                print(f"Skipping {df_name}: Missing columns {missing_columns}")
                continue

            before_dropping = df.shape[0]  # Count the number of rows before dropping duplicates
            df.drop_duplicates(subset=subset_columns, inplace=True)
            after_dropping = df.shape[0]  # Count the number of rows after dropping duplicates
            print(f"Dropped {before_dropping - after_dropping} duplicate rows from {df_name} based on columns: {subset_columns}")
        else:
            print(f"No duplicate check columns provided for {df_name}")

# Dictionary where the key is the DataFrame name, and the value is the subset of columns to check for duplicates
columns_to_check = {
    'city_weather': ['city_id', 'datetime'],
    'routes_weather': ['route_id', 'date'],
    'trucks_table': ['truck_id'],
    'drivers_table': ['driver_id'],
    'routes_table': ['route_id', 'destination_id', 'origin_id'],
    'truck_schedule_table': ['truck_id', 'route_id', 'departure_date']
}

# Call the function to drop duplicates from the DataFrames
drop_duplicates_from_dfs(dfs, columns_to_check)

# Check the result by printing the modified DataFrames
for df_name, df in dfs.items():
    print(f"\n{df_name} DataFrame after dropping duplicates:")
    print(df)


Dropped 707 duplicate rows from city_weather based on columns: ['city_id', 'datetime']
Dropped 0 duplicate rows from drivers_table based on columns: ['driver_id']
Dropped 0 duplicate rows from routes_table based on columns: ['route_id', 'destination_id', 'origin_id']
Dropped 0 duplicate rows from routes_weather based on columns: ['route_id', 'date']
No duplicate check columns provided for traffic_table
Dropped 0 duplicate rows from truck_schedule_table based on columns: ['truck_id', 'route_id', 'departure_date']
Dropped 0 duplicate rows from trucks_table based on columns: ['truck_id']

city_weather DataFrame after dropping duplicates:
          city_id  temp  wind_speed           description  precip  humidity  \
0      C-731988ba    52           1  Patchy rain possible     0.0        80   
1      C-841ebdcb    66           6         Partly cloudy     0.0        84   
2      C-ef47bdcd    10           6                 Sunny     0.0        57   
3      C-ff8c0c3c    34           8      

In [8]:
def drop_duplicates_from_dfs(dfs, columns_to_check):   
    for df_name, df in dfs.items():
        if df_name in columns_to_check:
            subset_columns = columns_to_check[df_name]
            
            # Check if all subset columns exist in the DataFrame
            missing_columns = [col for col in subset_columns if col not in df.columns]
            if missing_columns:
                print(f"Skipping {df_name}: Missing columns {missing_columns}")
                continue

            before_dropping = df.shape[0]  # Count the number of rows before dropping duplicates
            df.drop_duplicates(subset=subset_columns, inplace=True)
            after_dropping = df.shape[0]  # Count the number of rows after dropping duplicates
            print(f"Dropped {before_dropping - after_dropping} duplicate rows from {df_name} based on columns: {subset_columns}")
            
            # Print the number of rows and columns in each DataFrame after dropping duplicates
            print(f"{df_name} DataFrame after dropping duplicates: {df.shape[0]} rows, {df.shape[1]} columns")
        else:
            print(f"No duplicate check columns provided for {df_name}")

# Dictionary where the key is the DataFrame name, and the value is the subset of columns to check for duplicates
columns_to_check = {
    'city_weather': ['city_id', 'datetime'],
    'routes_weather': ['route_id', 'date'],
    'trucks_table': ['truck_id'],
    'drivers_table': ['driver_id'],
    'routes_table': ['route_id', 'destination_id', 'origin_id'],
    'truck_schedule_table': ['truck_id', 'route_id', 'departure_date']
}

# Call the function to drop duplicates from the DataFrames
drop_duplicates_from_dfs(dfs, columns_to_check)

# Check the result by printing the modified DataFrames' shapes
for df_name, df in dfs.items():
    print(f"\n{df_name} DataFrame has {df.shape[0]} rows and {df.shape[1]} columns after dropping duplicates.")


Dropped 0 duplicate rows from city_weather based on columns: ['city_id', 'datetime']
city_weather DataFrame after dropping duplicates: 38228 rows, 9 columns
Dropped 0 duplicate rows from drivers_table based on columns: ['driver_id']
drivers_table DataFrame after dropping duplicates: 1100 rows, 9 columns
Dropped 0 duplicate rows from routes_table based on columns: ['route_id', 'destination_id', 'origin_id']
routes_table DataFrame after dropping duplicates: 1896 rows, 5 columns
Dropped 0 duplicate rows from routes_weather based on columns: ['route_id', 'date']
routes_weather DataFrame after dropping duplicates: 396149 rows, 9 columns
No duplicate check columns provided for traffic_table
Dropped 0 duplicate rows from truck_schedule_table based on columns: ['truck_id', 'route_id', 'departure_date']
truck_schedule_table DataFrame after dropping duplicates: 10292 rows, 5 columns
Dropped 0 duplicate rows from trucks_table based on columns: ['truck_id']
trucks_table DataFrame after dropping du

FEATURE ENGINEERING

Merge Route_weather with Schedule data * create new data frame from scheduledf and add estimated_arrival and departure_date columns estimated_arriaval= df[estimated_arrival].dt.ceil("6H') departure_date= df[departure_date].dt.floor("6H")

In [9]:
truck_schedule_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10292 entries, 0 to 10291
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   truck_id           10292 non-null  int64              
 1   route_id           10292 non-null  object             
 2   departure_date     10292 non-null  datetime64[us, UTC]
 3   estimated_arrival  10292 non-null  datetime64[us, UTC]
 4   delay              10292 non-null  int64              
dtypes: datetime64[us, UTC](2), int64(2), object(1)
memory usage: 402.2+ KB


In [10]:
routes_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396149 entries, 0 to 396148
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   route_id     396149 non-null  object             
 1   date         396149 non-null  datetime64[us, UTC]
 2   temp         396149 non-null  int64              
 3   wind_speed   396149 non-null  int64              
 4   description  396149 non-null  object             
 5   precip       396149 non-null  float64            
 6   humidity     396149 non-null  int64              
 7   visibility   396149 non-null  int64              
 8   pressure     396149 non-null  int64              
dtypes: datetime64[us, UTC](1), float64(1), int64(5), object(2)
memory usage: 27.2+ MB


In [11]:
import pandas as pd

def process_dates(df, estimated_arrival_column, departure_date_column):
    # Function to convert to UTC if not already timezone-aware
    def convert_to_utc(series):
        if series.dt.tz is None:  # Check if the series is not timezone-aware
            return series.dt.tz_localize('UTC')
        else:
            return series.dt.tz_convert('UTC')  # Convert to UTC if it already has a timezone

    # Step 1: Create a copy of the DataFrame to avoid modifying the original
    df_copy = df.copy()

    # Step 2: Convert both columns to datetime format and set to UTC timezone
    df_copy[estimated_arrival_column] = convert_to_utc(pd.to_datetime(df_copy[estimated_arrival_column], errors='coerce'))
    df_copy[departure_date_column] = convert_to_utc(pd.to_datetime(df_copy[departure_date_column], errors='coerce'))

    # Step 3: Apply ceil operation to estimated arrival
    df_copy[estimated_arrival_column] = df_copy[estimated_arrival_column].dt.ceil("6H")

    # Step 4: Floor the 'departure_date' column to the nearest 6 hours
    df_copy[departure_date_column] = df_copy[departure_date_column].dt.floor("6H")

    # Step 5: Create a new column 'date' with date ranges between 'departure_date' and 'estimated_arrival'
    df_copy['date'] = [
        pd.date_range(start=row[departure_date_column], end=row[estimated_arrival_column], freq='6H')
        for index, row in df_copy.iterrows()
    ]
    
    # Step 6: Explode the 'date' column to separate rows for each date in the range
    df_copy = df_copy.explode('date').reset_index(drop=True)

    return df_copy


# Define target table(s) to process
target_tables = ['truck_schedule_table']  # Your actual table names from the 'dfs' dictionary
departure_date_column = 'departure_date'
estimated_arrival_column = 'estimated_arrival'

# Process the DataFrames
for table_name in list(dfs.keys()):  # Use list(dfs.keys()) to avoid changing the dictionary during iteration
    df = dfs[table_name]
    if table_name in target_tables:
        # Create a copy and process dates
        result_df_copy = process_dates(df, estimated_arrival_column, departure_date_column)
        
        # Merge with routes_weather on 'route_id' and 'date'
        scheduled_weather = pd.merge(result_df_copy, dfs['routes_weather'], on=['route_id', 'date'], how='left')

        # Display or log the merged DataFrame for review
        print(f"Processed {table_name} with route weather data merge:\n", scheduled_weather.head())

        # Save the processed DataFrame under a new key in the 'dfs' dictionary
        dfs[f'{table_name}_processed'] = scheduled_weather


Processed truck_schedule_table with route weather data merge:
    truck_id    route_id            departure_date         estimated_arrival  \
0  23244180  R-b1e8e9c8 2019-02-06 06:00:00+00:00 2019-02-07 06:00:00+00:00   
1  23244180  R-b1e8e9c8 2019-02-06 06:00:00+00:00 2019-02-07 06:00:00+00:00   
2  23244180  R-b1e8e9c8 2019-02-06 06:00:00+00:00 2019-02-07 06:00:00+00:00   
3  23244180  R-b1e8e9c8 2019-02-06 06:00:00+00:00 2019-02-07 06:00:00+00:00   
4  23244180  R-b1e8e9c8 2019-02-06 06:00:00+00:00 2019-02-07 06:00:00+00:00   

   delay                      date  temp  wind_speed        description  \
0      1 2019-02-06 06:00:00+00:00  52.0         7.0              Clear   
1      1 2019-02-06 12:00:00+00:00  37.0         4.0  Patchy light rain   
2      1 2019-02-06 18:00:00+00:00  36.0         4.0  Patchy light rain   
3      1 2019-02-07 00:00:00+00:00  52.0         7.0              Clear   
4      1 2019-02-07 06:00:00+00:00  52.0         7.0              Clear   

   precip  

Merge the resultant dataframe with route_weather on route_id and date (left)

In [12]:
scheduled_weather

,truck_id,route_id,departure_date,estimated_arrival,delay,date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,23244180,R-b1e8e9c8,2019-02-06 06:00:00+00:00,2019-02-07 06:00:00+00:00,1,2019-02-06 06:00:00+00:00,52.0,7.0,Clear,0.0,86.0,6.0,1020.0
1,23244180,R-b1e8e9c8,2019-02-06 06:00:00+00:00,2019-02-07 06:00:00+00:00,1,2019-02-06 12:00:00+00:00,37.0,4.0,Patchy light rain,0.0,99.0,6.0,1010.0
2,23244180,R-b1e8e9c8,2019-02-06 06:00:00+00:00,2019-02-07 06:00:00+00:00,1,2019-02-06 18:00:00+00:00,36.0,4.0,Patchy light rain,0.0,98.0,6.0,1010.0
3,23244180,R-b1e8e9c8,2019-02-06 06:00:00+00:00,2019-02-07 06:00:00+00:00,1,2019-02-07 00:00:00+00:00,52.0,7.0,Clear,0.0,85.0,6.0,1021.0
4,23244180,R-b1e8e9c8,2019-02-06 06:00:00+00:00,2019-02-07 06:00:00+00:00,1,2019-02-07 06:00:00+00:00,52.0,7.0,Clear,0.0,85.0,6.0,1021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59674,48204624,R-f5d9a32d,2019-01-15 06:00:00+00:00,2019-01-22 06:00:00+00:00,0,2019-01-22 06:00:00+00:00,30.0,14.0,Moderate snow,0.0,83.0,3.0,1011.0
59675,10787409,R-ef208446,2019-01-13 06:00:00+00:00,2019-01-14 00:00:00+00:00,0,2019-01-13 06:00:00+00:00,68.0,1.0,Clear,0.0,82.0,6.0,1017.0
59676,10787409,R-ef208446,2019-01-13 06:00:00+00:00,2019-01-14 00:00:00+00:00,0,2019-01-13 12:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59677,10787409,R-ef208446,2019-01-13 06:00:00+00:00,2019-01-14 00:00:00+00:00,0,2019-01-13 18:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
import numpy as np
# Define a custom function to calculate mode with error handling
def custom_mode(x):
    mode_values = x.mode()
    if len(mode_values) > 0:
        return mode_values.iloc[0]
    else:
        return np.nan  # or a default value, such as '' or np.nan

# Group by specified columns and aggregate
scheduled_weather_grp = scheduled_weather.groupby(['truck_id','route_id'], as_index=False).agg(
             route_avg_temp=('temp','mean'),
             route_avg_wind_speed=('wind_speed','mean'),
             route_avg_precip=('precip','mean'),
             route_avg_humidity=('humidity','mean'),
             route_avg_visibility=('visibility','mean'),
             route_avg_pressure=('pressure','mean'),
             route_description=('description', custom_mode)
         )

scheduled_weather_grp

,truck_id,route_id,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,route_avg_pressure,route_description
0,10008392,R-06dfe39e,50.750000,6.416667,0.000000,79.416667,5.833333,1015.250000,Partly cloudy
1,10008392,R-35dca729,56.600000,9.400000,0.000000,72.200000,5.700000,1013.300000,Partly cloudy
2,10008392,R-51d1a32a,46.636364,8.909091,0.000000,76.181818,6.000000,1016.818182,Partly cloudy
3,10008392,R-7ab5ae85,58.750000,7.000000,0.000000,82.250000,4.000000,1016.000000,Sunny
4,10008392,R-c6f8e036,47.916667,6.166667,0.000000,88.500000,5.583333,1011.000000,Clear
...,...,...,...,...,...,...,...,...,...
10229,54866249,R-2f7cdfa5,58.578947,12.000000,0.000000,60.526316,6.000000,1014.263158,Sunny
10230,54866249,R-43613017,52.916667,5.708333,0.000000,77.541667,6.000000,1015.083333,Clear
10231,54866249,R-e3ad4c7e,81.928571,11.428571,0.000000,69.500000,6.000000,1014.642857,Sunny
10232,55417825,R-92744e22,70.705882,6.323529,0.002941,71.176471,6.000000,1015.647059,Patchy rain possible


merge schedule df with schedule_weather merge

In [14]:
schedule_weather_merge=truck_schedule_table.merge(scheduled_weather_grp,on=['truck_id','route_id'],how='left')
schedule_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,route_avg_pressure,route_description
0,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:09:00+00:00,1,45.800000,5.800000,0.000000,90.600000,6.000000,1016.400000,Clear
1,10234289,R-d053f5bc,2019-01-22 07:00:00+00:00,2019-01-22 22:51:00+00:00,0,70.500000,6.000000,0.000000,78.000000,6.000000,1015.000000,Sunny
2,26739096,R-bf6ee666,2019-01-07 07:00:00+00:00,2019-01-07 15:54:36+00:00,1,38.666667,12.666667,0.000000,66.333333,3.333333,1005.666667,Blowing snow
3,20067958,R-abc9e78b,2019-01-04 07:00:00+00:00,2019-01-05 05:28:12+00:00,0,57.800000,2.200000,0.000000,78.800000,5.600000,1021.400000,Clear
4,23639934,R-7dac228a,2019-01-31 07:00:00+00:00,2019-01-31 13:40:12+00:00,0,57.666667,6.666667,0.000000,72.666667,6.000000,1016.666667,Partly cloudy
...,...,...,...,...,...,...,...,...,...,...,...,...
10287,30423832,R-f8b24663,2019-01-28 07:00:00+00:00,2019-01-28 23:47:24+00:00,1,70.000000,4.000000,0.000000,82.250000,4.500000,1011.750000,Patchy rain possible
10288,25117738,R-c172e39f,2019-01-01 07:00:00+00:00,2019-01-02 13:15:00+00:00,0,62.666667,11.000000,0.000000,79.000000,6.000000,1008.333333,Light rain shower
10289,11972697,R-b5a32b38,2019-01-19 07:00:00+00:00,2019-01-19 19:37:12+00:00,0,67.500000,9.250000,0.000000,60.000000,3.250000,1011.750000,Fog
10290,48204624,R-f5d9a32d,2019-01-15 07:00:00+00:00,2019-01-22 03:44:24+00:00,0,56.448276,10.206897,0.010345,61.655172,5.310345,1005.724138,Sunny


In [15]:
#take hourly as weather data available hourly

import pandas as pd

# Create a copy of the truck_schedule_table DataFrame
nearest_hour_schedule_df = truck_schedule_table.copy()

# Convert 'estimated_arrival' and 'departure_date' columns to datetime (if they aren't already)
nearest_hour_schedule_df['estimated_arrival'] = pd.to_datetime(nearest_hour_schedule_df['estimated_arrival'], errors='coerce')
nearest_hour_schedule_df['departure_date'] = pd.to_datetime(nearest_hour_schedule_df['departure_date'], errors='coerce')

# Round 'estimated_arrival' to the nearest hour
nearest_hour_schedule_df['estimated_arrival_nearest_hour'] = nearest_hour_schedule_df['estimated_arrival'].dt.round("H")

# Round 'departure_date' to the nearest hour
nearest_hour_schedule_df['departure_date_nearest_hour'] = nearest_hour_schedule_df['departure_date'].dt.round("H")

# Merge the result with routes_table on 'route_id'
nearest_hour_schedule_route_df = pd.merge(nearest_hour_schedule_df, routes_table, on='route_id', how='left')

# Display the resulting DataFrame
print(nearest_hour_schedule_route_df)


       truck_id    route_id            departure_date  \
0      23244180  R-b1e8e9c8 2019-02-06 07:00:00+00:00   
1      10234289  R-d053f5bc 2019-01-22 07:00:00+00:00   
2      26739096  R-bf6ee666 2019-01-07 07:00:00+00:00   
3      20067958  R-abc9e78b 2019-01-04 07:00:00+00:00   
4      23639934  R-7dac228a 2019-01-31 07:00:00+00:00   
...         ...         ...                       ...   
10287  30423832  R-f8b24663 2019-01-28 07:00:00+00:00   
10288  25117738  R-c172e39f 2019-01-01 07:00:00+00:00   
10289  11972697  R-b5a32b38 2019-01-19 07:00:00+00:00   
10290  48204624  R-f5d9a32d 2019-01-15 07:00:00+00:00   
10291  10787409  R-ef208446 2019-01-13 07:00:00+00:00   

              estimated_arrival  delay estimated_arrival_nearest_hour  \
0     2019-02-07 04:09:00+00:00      1      2019-02-07 04:00:00+00:00   
1     2019-01-22 22:51:00+00:00      0      2019-01-22 23:00:00+00:00   
2     2019-01-07 15:54:36+00:00      1      2019-01-07 16:00:00+00:00   
3     2019-01-05 05:28:

In [16]:
# Assume 'city_weather' is your original DataFrame
origin_weather_data= city_weather.copy()  # First copy
destination_weather_data = city_weather.copy()  # Second copy

In [17]:
nearest_hour_schedule_route_df.head()

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours
0,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:09:00+00:00,1,2019-02-07 04:00:00+00:00,2019-02-06 07:00:00+00:00,C-84f378bb,C-2aaf0e1a,1057.44,21.15
1,10234289,R-d053f5bc,2019-01-22 07:00:00+00:00,2019-01-22 22:51:00+00:00,0,2019-01-22 23:00:00+00:00,2019-01-22 07:00:00+00:00,C-d9e9d934,C-fc66f0ab,792.60,15.85
2,26739096,R-bf6ee666,2019-01-07 07:00:00+00:00,2019-01-07 15:54:36+00:00,1,2019-01-07 16:00:00+00:00,2019-01-07 07:00:00+00:00,C-d80a1e7d,C-9bbf5d8a,445.41,8.91
3,20067958,R-abc9e78b,2019-01-04 07:00:00+00:00,2019-01-05 05:28:12+00:00,0,2019-01-05 05:00:00+00:00,2019-01-04 07:00:00+00:00,C-2aaf0e1a,C-2c9e75ef,1123.72,22.47
4,23639934,R-7dac228a,2019-01-31 07:00:00+00:00,2019-01-31 13:40:12+00:00,0,2019-01-31 14:00:00+00:00,2019-01-31 07:00:00+00:00,C-280b55fb,C-fc66f0ab,333.35,6.67


In [18]:
origin_weather_data

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,C-731988ba,52,1,Patchy rain possible,0.0,80,6,1015,2019-02-14 05:00:00+00:00
1,C-841ebdcb,66,6,Partly cloudy,0.0,84,6,1020,2019-01-07 20:00:00+00:00
2,C-ef47bdcd,10,6,Sunny,0.0,57,6,1036,2019-02-08 06:00:00+00:00
3,C-ff8c0c3c,34,8,Overcast,0.0,100,6,1024,2019-01-15 13:00:00+00:00
4,C-3dbd8b2e,41,11,Partly cloudy,0.0,73,6,1021,2019-01-07 15:00:00+00:00
...,...,...,...,...,...,...,...,...,...
38930,C-4fe0fa24,41,11,Partly cloudy,0.0,70,6,1022,2019-02-02 20:00:00+00:00
38931,C-56b68559,25,13,Overcast,0.0,79,6,1011,2019-02-04 14:00:00+00:00
38932,C-2c9e75ef,36,10,Clear,0.0,59,6,1023,2019-01-20 19:00:00+00:00
38933,C-03bb3e48,-9,8,Cloudy,0.0,88,6,1039,2019-01-01 09:00:00+00:00


            * create a copy of the 'weather_df' DataFrame for manipulation
            * Drop the 'date' and 'hour' columns from 'origin_weather_data'
            * Create a copy of the 'weather_df' DataFrame for manipulation
            * Drop the 'date' and 'hour' columns from 'destination_weather_data'
            * Merge 'nearest_hour_schedule_route_df' with 'origin_weather_data' based on specified columns


In [19]:
#Merge 'nearest_hour_schedule_route_df' with 'origin_weather_data' based on specified columns

# Ensure both DataFrames have the datetime columns in UTC format

# Check if departure_date_nearest_hour is timezone-naive
if nearest_hour_schedule_route_df['departure_date_nearest_hour'].dt.tz is None:
    # Localize to UTC if naive
    nearest_hour_schedule_route_df['departure_date_nearest_hour'] = nearest_hour_schedule_route_df['departure_date_nearest_hour'].dt.tz_localize('UTC')

# Check if datetime is timezone-naive
if origin_weather_data['datetime'].dt.tz is None:
    # Localize to UTC if naive
    origin_weather_data['datetime'] = pd.to_datetime(origin_weather_data['datetime'], errors='coerce').dt.tz_localize('UTC')

# Specify the columns to merge on
left_merge_columns = ['origin_id', 'departure_date_nearest_hour']  # Columns from nearest_hour_schedule_route_df
right_merge_columns = ['city_id', 'datetime']  # Columns from origin_weather_data

# Merge the DataFrames
origin_weather_merge = pd.merge(
    nearest_hour_schedule_route_df, 
    origin_weather_data, 
    left_on=left_merge_columns, 
    right_on=right_merge_columns, 
    how='left'
)

# Display the resulting merged DataFrame
origin_weather_merge


,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:09:00+00:00,1,2019-02-07 04:00:00+00:00,2019-02-06 07:00:00+00:00,C-84f378bb,C-2aaf0e1a,1057.44,21.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,10234289,R-d053f5bc,2019-01-22 07:00:00+00:00,2019-01-22 22:51:00+00:00,0,2019-01-22 23:00:00+00:00,2019-01-22 07:00:00+00:00,C-d9e9d934,C-fc66f0ab,792.60,15.85,C-d9e9d934,27.0,7.0,Sunny,0.0,59.0,6.0,1040.0,2019-01-22 07:00:00+00:00
2,26739096,R-bf6ee666,2019-01-07 07:00:00+00:00,2019-01-07 15:54:36+00:00,1,2019-01-07 16:00:00+00:00,2019-01-07 07:00:00+00:00,C-d80a1e7d,C-9bbf5d8a,445.41,8.91,C-d80a1e7d,46.0,6.0,Partly cloudy,0.0,85.0,6.0,1015.0,2019-01-07 07:00:00+00:00
3,20067958,R-abc9e78b,2019-01-04 07:00:00+00:00,2019-01-05 05:28:12+00:00,0,2019-01-05 05:00:00+00:00,2019-01-04 07:00:00+00:00,C-2aaf0e1a,C-2c9e75ef,1123.72,22.47,C-2aaf0e1a,32.0,4.0,Sunny,0.0,79.0,6.0,1016.0,2019-01-04 07:00:00+00:00
4,23639934,R-7dac228a,2019-01-31 07:00:00+00:00,2019-01-31 13:40:12+00:00,0,2019-01-31 14:00:00+00:00,2019-01-31 07:00:00+00:00,C-280b55fb,C-fc66f0ab,333.35,6.67,C-280b55fb,-6.0,10.0,Partly cloudy,0.0,59.0,6.0,1032.0,2019-01-31 07:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,30423832,R-f8b24663,2019-01-28 07:00:00+00:00,2019-01-28 23:47:24+00:00,1,2019-01-29 00:00:00+00:00,2019-01-28 07:00:00+00:00,C-d9e9d934,C-19236709,839.30,16.79,C-d9e9d934,39.0,9.0,Overcast,0.0,68.0,6.0,1016.0,2019-01-28 07:00:00+00:00
10288,25117738,R-c172e39f,2019-01-01 07:00:00+00:00,2019-01-02 13:15:00+00:00,0,2019-01-02 13:00:00+00:00,2019-01-01 07:00:00+00:00,C-3dbd8b2e,C-7212cebe,1512.50,30.25,C-3dbd8b2e,37.0,1.0,Sunny,0.0,54.0,6.0,1030.0,2019-01-01 07:00:00+00:00
10289,11972697,R-b5a32b38,2019-01-19 07:00:00+00:00,2019-01-19 19:37:12+00:00,0,2019-01-19 20:00:00+00:00,2019-01-19 07:00:00+00:00,C-e5bfb4e5,C-84f378bb,631.07,12.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
10290,48204624,R-f5d9a32d,2019-01-15 07:00:00+00:00,2019-01-22 03:44:24+00:00,0,2019-01-22 04:00:00+00:00,2019-01-15 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [20]:
destination_weather_data

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,C-731988ba,52,1,Patchy rain possible,0.0,80,6,1015,2019-02-14 05:00:00+00:00
1,C-841ebdcb,66,6,Partly cloudy,0.0,84,6,1020,2019-01-07 20:00:00+00:00
2,C-ef47bdcd,10,6,Sunny,0.0,57,6,1036,2019-02-08 06:00:00+00:00
3,C-ff8c0c3c,34,8,Overcast,0.0,100,6,1024,2019-01-15 13:00:00+00:00
4,C-3dbd8b2e,41,11,Partly cloudy,0.0,73,6,1021,2019-01-07 15:00:00+00:00
...,...,...,...,...,...,...,...,...,...
38930,C-4fe0fa24,41,11,Partly cloudy,0.0,70,6,1022,2019-02-02 20:00:00+00:00
38931,C-56b68559,25,13,Overcast,0.0,79,6,1011,2019-02-04 14:00:00+00:00
38932,C-2c9e75ef,36,10,Clear,0.0,59,6,1023,2019-01-20 19:00:00+00:00
38933,C-03bb3e48,-9,8,Cloudy,0.0,88,6,1039,2019-01-01 09:00:00+00:00


Merge 'origin_weather_merge' with 'destination_weather_data' based on specified columns


In [21]:
#Merge 'origin_weather_merge' with 'destination_weather_data' based on specified columns

# Ensure both DataFrames have the datetime columns in UTC format

# Check if estimated_arrival_nearest_hour is timezone-naive
if origin_weather_merge['estimated_arrival_nearest_hour'].dt.tz is None:
    # Localize to UTC if naive
    origin_weather_merge['estimated_arrival_nearest_hour'] = origin_weather_merge['estimated_arrival_nearest_hour'].dt.tz_localize('UTC')

# Check if datetime is timezone-naive
if destination_weather_data['datetime'].dt.tz is None:
    # Localize to UTC if naive
    destination_weather_data['datetime'] = pd.to_datetime(destination_weather_data['datetime'], errors='coerce').dt.tz_localize('UTC')

# Specify the columns to merge on
left_merge_columns = ['destination_id', 'estimated_arrival_nearest_hour']  # Columns from origin_weather_merge
right_merge_columns = ['city_id', 'datetime']  # Columns from destination_weather_data

# Merge the DataFrames
schedule_data_merge = pd.merge(
    origin_weather_merge, 
    destination_weather_data, 
    left_on=left_merge_columns, 
    right_on=right_merge_columns, 
    how='left'
)

# Display the resulting merged DataFrame
schedule_data_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,datetime_x,city_id_y,temp_y,wind_speed_y,description_y,precip_y,humidity_y,visibility_y,pressure_y,datetime_y
0,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:09:00+00:00,1,2019-02-07 04:00:00+00:00,2019-02-06 07:00:00+00:00,C-84f378bb,C-2aaf0e1a,1057.44,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,10234289,R-d053f5bc,2019-01-22 07:00:00+00:00,2019-01-22 22:51:00+00:00,0,2019-01-22 23:00:00+00:00,2019-01-22 07:00:00+00:00,C-d9e9d934,C-fc66f0ab,792.60,...,2019-01-22 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,26739096,R-bf6ee666,2019-01-07 07:00:00+00:00,2019-01-07 15:54:36+00:00,1,2019-01-07 16:00:00+00:00,2019-01-07 07:00:00+00:00,C-d80a1e7d,C-9bbf5d8a,445.41,...,2019-01-07 07:00:00+00:00,C-9bbf5d8a,46.0,4.0,Partly cloudy,0.0,64.0,6.0,1027.0,2019-01-07 16:00:00+00:00
3,20067958,R-abc9e78b,2019-01-04 07:00:00+00:00,2019-01-05 05:28:12+00:00,0,2019-01-05 05:00:00+00:00,2019-01-04 07:00:00+00:00,C-2aaf0e1a,C-2c9e75ef,1123.72,...,2019-01-04 07:00:00+00:00,C-2c9e75ef,39.0,6.0,Clear,0.0,78.0,6.0,1018.0,2019-01-05 05:00:00+00:00
4,23639934,R-7dac228a,2019-01-31 07:00:00+00:00,2019-01-31 13:40:12+00:00,0,2019-01-31 14:00:00+00:00,2019-01-31 07:00:00+00:00,C-280b55fb,C-fc66f0ab,333.35,...,2019-01-31 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,30423832,R-f8b24663,2019-01-28 07:00:00+00:00,2019-01-28 23:47:24+00:00,1,2019-01-29 00:00:00+00:00,2019-01-28 07:00:00+00:00,C-d9e9d934,C-19236709,839.30,...,2019-01-28 07:00:00+00:00,C-19236709,7.0,16.0,Light snow,0.0,86.0,6.0,1023.0,2019-01-29 00:00:00+00:00
10288,25117738,R-c172e39f,2019-01-01 07:00:00+00:00,2019-01-02 13:15:00+00:00,0,2019-01-02 13:00:00+00:00,2019-01-01 07:00:00+00:00,C-3dbd8b2e,C-7212cebe,1512.50,...,2019-01-01 07:00:00+00:00,C-7212cebe,21.0,7.0,Sunny,0.0,71.0,6.0,1017.0,2019-01-02 13:00:00+00:00
10289,11972697,R-b5a32b38,2019-01-19 07:00:00+00:00,2019-01-19 19:37:12+00:00,0,2019-01-19 20:00:00+00:00,2019-01-19 07:00:00+00:00,C-e5bfb4e5,C-84f378bb,631.07,...,NaT,C-84f378bb,23.0,10.0,Clear,0.0,68.0,6.0,1030.0,2019-01-19 20:00:00+00:00
10290,48204624,R-f5d9a32d,2019-01-15 07:00:00+00:00,2019-01-22 03:44:24+00:00,0,2019-01-22 04:00:00+00:00,2019-01-15 07:00:00+00:00,NaN,NaN,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [22]:
# Create a copy of the schedule_data_merge DataFrame
schedule_data_copy = schedule_data_merge.copy()
schedule_data_copy

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,datetime_x,city_id_y,temp_y,wind_speed_y,description_y,precip_y,humidity_y,visibility_y,pressure_y,datetime_y
0,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:09:00+00:00,1,2019-02-07 04:00:00+00:00,2019-02-06 07:00:00+00:00,C-84f378bb,C-2aaf0e1a,1057.44,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,10234289,R-d053f5bc,2019-01-22 07:00:00+00:00,2019-01-22 22:51:00+00:00,0,2019-01-22 23:00:00+00:00,2019-01-22 07:00:00+00:00,C-d9e9d934,C-fc66f0ab,792.60,...,2019-01-22 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,26739096,R-bf6ee666,2019-01-07 07:00:00+00:00,2019-01-07 15:54:36+00:00,1,2019-01-07 16:00:00+00:00,2019-01-07 07:00:00+00:00,C-d80a1e7d,C-9bbf5d8a,445.41,...,2019-01-07 07:00:00+00:00,C-9bbf5d8a,46.0,4.0,Partly cloudy,0.0,64.0,6.0,1027.0,2019-01-07 16:00:00+00:00
3,20067958,R-abc9e78b,2019-01-04 07:00:00+00:00,2019-01-05 05:28:12+00:00,0,2019-01-05 05:00:00+00:00,2019-01-04 07:00:00+00:00,C-2aaf0e1a,C-2c9e75ef,1123.72,...,2019-01-04 07:00:00+00:00,C-2c9e75ef,39.0,6.0,Clear,0.0,78.0,6.0,1018.0,2019-01-05 05:00:00+00:00
4,23639934,R-7dac228a,2019-01-31 07:00:00+00:00,2019-01-31 13:40:12+00:00,0,2019-01-31 14:00:00+00:00,2019-01-31 07:00:00+00:00,C-280b55fb,C-fc66f0ab,333.35,...,2019-01-31 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,30423832,R-f8b24663,2019-01-28 07:00:00+00:00,2019-01-28 23:47:24+00:00,1,2019-01-29 00:00:00+00:00,2019-01-28 07:00:00+00:00,C-d9e9d934,C-19236709,839.30,...,2019-01-28 07:00:00+00:00,C-19236709,7.0,16.0,Light snow,0.0,86.0,6.0,1023.0,2019-01-29 00:00:00+00:00
10288,25117738,R-c172e39f,2019-01-01 07:00:00+00:00,2019-01-02 13:15:00+00:00,0,2019-01-02 13:00:00+00:00,2019-01-01 07:00:00+00:00,C-3dbd8b2e,C-7212cebe,1512.50,...,2019-01-01 07:00:00+00:00,C-7212cebe,21.0,7.0,Sunny,0.0,71.0,6.0,1017.0,2019-01-02 13:00:00+00:00
10289,11972697,R-b5a32b38,2019-01-19 07:00:00+00:00,2019-01-19 19:37:12+00:00,0,2019-01-19 20:00:00+00:00,2019-01-19 07:00:00+00:00,C-e5bfb4e5,C-84f378bb,631.07,...,NaT,C-84f378bb,23.0,10.0,Clear,0.0,68.0,6.0,1030.0,2019-01-19 20:00:00+00:00
10290,48204624,R-f5d9a32d,2019-01-15 07:00:00+00:00,2019-01-22 03:44:24+00:00,0,2019-01-22 04:00:00+00:00,2019-01-15 07:00:00+00:00,NaN,NaN,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [23]:
# Round 'estimated_arrival' to the nearest hour
schedule_data_copy['estimated_arrival'] = schedule_data_copy['estimated_arrival'].dt.round('H')

# Round 'departure_date' to the nearest hour
schedule_data_copy['departure_date'] = schedule_data_copy['departure_date'].dt.round('H')

# Display the modified DataFrame
schedule_data_copy[['estimated_arrival', 'departure_date']].head()

nearest_hour_schedule_df = schedule_data_copy
nearest_hour_schedule_df


,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,datetime_x,city_id_y,temp_y,wind_speed_y,description_y,precip_y,humidity_y,visibility_y,pressure_y,datetime_y
0,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:00:00+00:00,1,2019-02-07 04:00:00+00:00,2019-02-06 07:00:00+00:00,C-84f378bb,C-2aaf0e1a,1057.44,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,10234289,R-d053f5bc,2019-01-22 07:00:00+00:00,2019-01-22 23:00:00+00:00,0,2019-01-22 23:00:00+00:00,2019-01-22 07:00:00+00:00,C-d9e9d934,C-fc66f0ab,792.60,...,2019-01-22 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,26739096,R-bf6ee666,2019-01-07 07:00:00+00:00,2019-01-07 16:00:00+00:00,1,2019-01-07 16:00:00+00:00,2019-01-07 07:00:00+00:00,C-d80a1e7d,C-9bbf5d8a,445.41,...,2019-01-07 07:00:00+00:00,C-9bbf5d8a,46.0,4.0,Partly cloudy,0.0,64.0,6.0,1027.0,2019-01-07 16:00:00+00:00
3,20067958,R-abc9e78b,2019-01-04 07:00:00+00:00,2019-01-05 05:00:00+00:00,0,2019-01-05 05:00:00+00:00,2019-01-04 07:00:00+00:00,C-2aaf0e1a,C-2c9e75ef,1123.72,...,2019-01-04 07:00:00+00:00,C-2c9e75ef,39.0,6.0,Clear,0.0,78.0,6.0,1018.0,2019-01-05 05:00:00+00:00
4,23639934,R-7dac228a,2019-01-31 07:00:00+00:00,2019-01-31 14:00:00+00:00,0,2019-01-31 14:00:00+00:00,2019-01-31 07:00:00+00:00,C-280b55fb,C-fc66f0ab,333.35,...,2019-01-31 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,30423832,R-f8b24663,2019-01-28 07:00:00+00:00,2019-01-29 00:00:00+00:00,1,2019-01-29 00:00:00+00:00,2019-01-28 07:00:00+00:00,C-d9e9d934,C-19236709,839.30,...,2019-01-28 07:00:00+00:00,C-19236709,7.0,16.0,Light snow,0.0,86.0,6.0,1023.0,2019-01-29 00:00:00+00:00
10288,25117738,R-c172e39f,2019-01-01 07:00:00+00:00,2019-01-02 13:00:00+00:00,0,2019-01-02 13:00:00+00:00,2019-01-01 07:00:00+00:00,C-3dbd8b2e,C-7212cebe,1512.50,...,2019-01-01 07:00:00+00:00,C-7212cebe,21.0,7.0,Sunny,0.0,71.0,6.0,1017.0,2019-01-02 13:00:00+00:00
10289,11972697,R-b5a32b38,2019-01-19 07:00:00+00:00,2019-01-19 20:00:00+00:00,0,2019-01-19 20:00:00+00:00,2019-01-19 07:00:00+00:00,C-e5bfb4e5,C-84f378bb,631.07,...,NaT,C-84f378bb,23.0,10.0,Clear,0.0,68.0,6.0,1030.0,2019-01-19 20:00:00+00:00
10290,48204624,R-f5d9a32d,2019-01-15 07:00:00+00:00,2019-01-22 04:00:00+00:00,0,2019-01-22 04:00:00+00:00,2019-01-15 07:00:00+00:00,NaN,NaN,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [24]:
# Function to ensure datetime columns are in UTC
def ensure_utc(column):
    if column.dt.tz is None:  # Check if the column is naive
        return column.dt.tz_localize('UTC')  # Localize to UTC
    else:
        return column.dt.tz_convert('UTC')  # Convert to UTC if already aware

# Ensure both datetime columns are in UTC
nearest_hour_schedule_df['departure_date'] = ensure_utc(nearest_hour_schedule_df['departure_date'])
nearest_hour_schedule_df['estimated_arrival'] = ensure_utc(nearest_hour_schedule_df['estimated_arrival'])

# Now create the custom date ranges
hourly_exploded_scheduled_df = (
    nearest_hour_schedule_df.assign(
        custom_date=[
            pd.date_range(start, end, freq='H')  # Create custom date ranges
            for start, end in zip(
                nearest_hour_schedule_df['departure_date'],
                nearest_hour_schedule_df['estimated_arrival']
            )  # Using departure and estimated arrival times
        ]
    ).explode('custom_date', ignore_index=True)  # Explode the DataFrame based on the custom date range
)

# Display the resulting DataFrame
hourly_exploded_scheduled_df[['departure_date', 'estimated_arrival', 'custom_date']].head()


,departure_date,estimated_arrival,custom_date
0,2019-02-06 07:00:00+00:00,2019-02-07 04:00:00+00:00,2019-02-06 07:00:00+00:00
1,2019-02-06 07:00:00+00:00,2019-02-07 04:00:00+00:00,2019-02-06 08:00:00+00:00
2,2019-02-06 07:00:00+00:00,2019-02-07 04:00:00+00:00,2019-02-06 09:00:00+00:00
3,2019-02-06 07:00:00+00:00,2019-02-07 04:00:00+00:00,2019-02-06 10:00:00+00:00
4,2019-02-06 07:00:00+00:00,2019-02-07 04:00:00+00:00,2019-02-06 11:00:00+00:00


In [25]:
traffic_table

,route_id,no_of_vehicles,accident,datetime
0,R-0f705429,2343.0,0,2019-02-06 20:00:00+00:00
1,R-f8a25f87,552.0,0,2019-01-20 19:00:00+00:00
2,R-3b99b000,645.0,0,2019-01-31 04:00:00+00:00
3,R-ac453ce8,621.0,0,2019-01-12 00:00:00+00:00
4,R-af1d28da,2916.0,0,2019-01-06 10:00:00+00:00
...,...,...,...,...
2597908,R-52e3a448,531.0,0,2019-01-15 02:00:00+00:00
2597909,R-a9ddb09f,2505.0,0,2019-01-19 10:00:00+00:00
2597910,R-933965b8,2379.0,1,2019-02-13 18:00:00+00:00
2597911,R-cda66d11,517.0,0,2019-01-16 04:00:00+00:00


In [26]:
hourly_exploded_scheduled_df

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,city_id_y,temp_y,wind_speed_y,description_y,precip_y,humidity_y,visibility_y,pressure_y,datetime_y,custom_date
0,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:00:00+00:00,1,2019-02-07 04:00:00+00:00,2019-02-06 07:00:00+00:00,C-84f378bb,C-2aaf0e1a,1057.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2019-02-06 07:00:00+00:00
1,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:00:00+00:00,1,2019-02-07 04:00:00+00:00,2019-02-06 07:00:00+00:00,C-84f378bb,C-2aaf0e1a,1057.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2019-02-06 08:00:00+00:00
2,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:00:00+00:00,1,2019-02-07 04:00:00+00:00,2019-02-06 07:00:00+00:00,C-84f378bb,C-2aaf0e1a,1057.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2019-02-06 09:00:00+00:00
3,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:00:00+00:00,1,2019-02-07 04:00:00+00:00,2019-02-06 07:00:00+00:00,C-84f378bb,C-2aaf0e1a,1057.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2019-02-06 10:00:00+00:00
4,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:00:00+00:00,1,2019-02-07 04:00:00+00:00,2019-02-06 07:00:00+00:00,C-84f378bb,C-2aaf0e1a,1057.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2019-02-06 11:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265410,10787409,R-ef208446,2019-01-13 07:00:00+00:00,2019-01-13 21:00:00+00:00,0,2019-01-13 21:00:00+00:00,2019-01-13 07:00:00+00:00,C-6ea51d66,C-594514f8,694.98,...,C-594514f8,25.0,2.0,Clear,0.0,78.0,6.0,1031.0,2019-01-13 21:00:00+00:00,2019-01-13 17:00:00+00:00
265411,10787409,R-ef208446,2019-01-13 07:00:00+00:00,2019-01-13 21:00:00+00:00,0,2019-01-13 21:00:00+00:00,2019-01-13 07:00:00+00:00,C-6ea51d66,C-594514f8,694.98,...,C-594514f8,25.0,2.0,Clear,0.0,78.0,6.0,1031.0,2019-01-13 21:00:00+00:00,2019-01-13 18:00:00+00:00
265412,10787409,R-ef208446,2019-01-13 07:00:00+00:00,2019-01-13 21:00:00+00:00,0,2019-01-13 21:00:00+00:00,2019-01-13 07:00:00+00:00,C-6ea51d66,C-594514f8,694.98,...,C-594514f8,25.0,2.0,Clear,0.0,78.0,6.0,1031.0,2019-01-13 21:00:00+00:00,2019-01-13 19:00:00+00:00
265413,10787409,R-ef208446,2019-01-13 07:00:00+00:00,2019-01-13 21:00:00+00:00,0,2019-01-13 21:00:00+00:00,2019-01-13 07:00:00+00:00,C-6ea51d66,C-594514f8,694.98,...,C-594514f8,25.0,2.0,Clear,0.0,78.0,6.0,1031.0,2019-01-13 21:00:00+00:00,2019-01-13 20:00:00+00:00


In [27]:
# Merge the traffic_table and hourly_exploded_scheduled_df DataFrames
scheduled_traffic = hourly_exploded_scheduled_df.merge(
    traffic_table,
    left_on=['route_id', 'custom_date'],  # Columns from hourly_exploded_scheduled_df
    right_on=['route_id', 'datetime'],        # Corresponding columns from traffic_table
    how='left'
)

# Display the resulting merged DataFrame
scheduled_traffic.head()


,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,description_y,precip_y,humidity_y,visibility_y,pressure_y,datetime_y,custom_date,no_of_vehicles,accident,datetime
0,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:00:00+00:00,1,2019-02-07 04:00:00+00:00,2019-02-06 07:00:00+00:00,C-84f378bb,C-2aaf0e1a,1057.44,...,NaN,NaN,NaN,NaN,NaN,NaT,2019-02-06 07:00:00+00:00,609.0,0,2019-02-06 07:00:00+00:00
1,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:00:00+00:00,1,2019-02-07 04:00:00+00:00,2019-02-06 07:00:00+00:00,C-84f378bb,C-2aaf0e1a,1057.44,...,NaN,NaN,NaN,NaN,NaN,NaT,2019-02-06 08:00:00+00:00,2449.0,0,2019-02-06 08:00:00+00:00
2,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:00:00+00:00,1,2019-02-07 04:00:00+00:00,2019-02-06 07:00:00+00:00,C-84f378bb,C-2aaf0e1a,1057.44,...,NaN,NaN,NaN,NaN,NaN,NaT,2019-02-06 09:00:00+00:00,1833.0,0,2019-02-06 09:00:00+00:00
3,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:00:00+00:00,1,2019-02-07 04:00:00+00:00,2019-02-06 07:00:00+00:00,C-84f378bb,C-2aaf0e1a,1057.44,...,NaN,NaN,NaN,NaN,NaN,NaT,2019-02-06 10:00:00+00:00,2614.0,0,2019-02-06 10:00:00+00:00
4,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:00:00+00:00,1,2019-02-07 04:00:00+00:00,2019-02-06 07:00:00+00:00,C-84f378bb,C-2aaf0e1a,1057.44,...,NaN,NaN,NaN,NaN,NaN,NaT,2019-02-06 11:00:00+00:00,2371.0,0,2019-02-06 11:00:00+00:00


In [28]:
#Define a custom aggregation function for accidents
def custom_agg(values):
    """
    Custom aggregation function to determine if any value in a group is 1 (indicating an accident).
    
    Args:
    values (iterable): Iterable of values in a group.
    
    Returns:
    int: 1 if any value is 1, else 0.
    """
    # Check if any value in the iterable is equal to 1
    return 1 if 1 in values else 0

# Group by 'unique_id', 'truck_id', and 'route_id', and apply custom aggregation
scheduled_route_traffic = scheduled_traffic.groupby([ 'truck_id', 'route_id'], as_index=False).agg(
    avg_no_of_vehicles=('no_of_vehicles', 'mean'),
    accident=('accident', custom_agg)
)

# Display the resulting DataFrame
scheduled_route_traffic.head()

,truck_id,route_id,avg_no_of_vehicles,accident
0,10008392,R-06dfe39e,1882.655738,0
1,10008392,R-35dca729,1820.098039,0
2,10008392,R-51d1a32a,1545.450000,0
3,10008392,R-7ab5ae85,1927.825000,0
4,10008392,R-c6f8e036,861.131148,0


Merge data

In [29]:
#renaming the schedule_data_merge with origin_destination_weather
origin_destination_weather=schedule_data_merge

In [30]:
#Merge schedule_route_traffic with origin_destination_weather

origin_destination_weather_traffic_merge=origin_destination_weather.merge(scheduled_route_traffic,on=['truck_id','route_id'],how='left')

Merge weather and Traffic

In [31]:
# Convert 'departure_date' to datetime in the schedule_weather_merge DataFrame
schedule_weather_merge['departure_date'] = pd.to_datetime(schedule_weather_merge['departure_date'], errors='coerce')

# If origin_destination_weather_traffic_merge has 'departure_date' as object, convert it as well
origin_destination_weather_traffic_merge['departure_date'] = pd.to_datetime(origin_destination_weather_traffic_merge['departure_date'], errors='coerce')


In [32]:
schedule_weather_merge.dtypes

truck_id                              int64
route_id                             object
departure_date          datetime64[us, UTC]
estimated_arrival       datetime64[us, UTC]
delay                                 int64
route_avg_temp                      float64
route_avg_wind_speed                float64
route_avg_precip                    float64
route_avg_humidity                  float64
route_avg_visibility                float64
route_avg_pressure                  float64
route_description                    object
dtype: object

In [33]:
origin_destination_weather_traffic_merge.dtypes

truck_id                                        int64
route_id                                       object
departure_date                    datetime64[us, UTC]
estimated_arrival                 datetime64[us, UTC]
delay                                           int64
estimated_arrival_nearest_hour    datetime64[us, UTC]
departure_date_nearest_hour       datetime64[us, UTC]
origin_id                                      object
destination_id                                 object
distance                                      float64
average_hours                                 float64
city_id_x                                      object
temp_x                                        float64
wind_speed_x                                  float64
description_x                                  object
precip_x                                      float64
humidity_x                                    float64
visibility_x                                  float64
pressure_x                  

In [34]:
#merged_data_weather_traffic=pd.merge(schedule_weather_merge, origin_destination_weather_traffic_merge, on=['truck_id', 'route_id', 'departure_date','estimated_arrival', 'delay'], how='left')

# Merge the DataFrames
merged_data_weather_traffic = pd.merge(
    schedule_weather_merge, 
    origin_destination_weather_traffic_merge, 
    on=['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay'], 
    how='left'
)

# Display the merged DataFrame
merged_data_weather_traffic.head()


,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,...,temp_y,wind_speed_y,description_y,precip_y,humidity_y,visibility_y,pressure_y,datetime_y,avg_no_of_vehicles,accident
0,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:09:00+00:00,1,45.800000,5.800000,0.0,90.600000,6.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1663.727273,1
1,10234289,R-d053f5bc,2019-01-22 07:00:00+00:00,2019-01-22 22:51:00+00:00,0,70.500000,6.000000,0.0,78.000000,6.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2269.352941,0
2,26739096,R-bf6ee666,2019-01-07 07:00:00+00:00,2019-01-07 15:54:36+00:00,1,38.666667,12.666667,0.0,66.333333,3.333333,...,46.0,4.0,Partly cloudy,0.0,64.0,6.0,1027.0,2019-01-07 16:00:00+00:00,1579.500000,0
3,20067958,R-abc9e78b,2019-01-04 07:00:00+00:00,2019-01-05 05:28:12+00:00,0,57.800000,2.200000,0.0,78.800000,5.600000,...,39.0,6.0,Clear,0.0,78.0,6.0,1018.0,2019-01-05 05:00:00+00:00,1552.391304,0
4,23639934,R-7dac228a,2019-01-31 07:00:00+00:00,2019-01-31 13:40:12+00:00,0,57.666667,6.666667,0.0,72.666667,6.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2259.875000,0


In [35]:
merged_data_weather_traffic

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,...,temp_y,wind_speed_y,description_y,precip_y,humidity_y,visibility_y,pressure_y,datetime_y,avg_no_of_vehicles,accident
0,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:09:00+00:00,1,45.800000,5.800000,0.000000,90.600000,6.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1663.727273,1
1,10234289,R-d053f5bc,2019-01-22 07:00:00+00:00,2019-01-22 22:51:00+00:00,0,70.500000,6.000000,0.000000,78.000000,6.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2269.352941,0
2,26739096,R-bf6ee666,2019-01-07 07:00:00+00:00,2019-01-07 15:54:36+00:00,1,38.666667,12.666667,0.000000,66.333333,3.333333,...,46.0,4.0,Partly cloudy,0.0,64.0,6.0,1027.0,2019-01-07 16:00:00+00:00,1579.500000,0
3,20067958,R-abc9e78b,2019-01-04 07:00:00+00:00,2019-01-05 05:28:12+00:00,0,57.800000,2.200000,0.000000,78.800000,5.600000,...,39.0,6.0,Clear,0.0,78.0,6.0,1018.0,2019-01-05 05:00:00+00:00,1552.391304,0
4,23639934,R-7dac228a,2019-01-31 07:00:00+00:00,2019-01-31 13:40:12+00:00,0,57.666667,6.666667,0.000000,72.666667,6.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2259.875000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,30423832,R-f8b24663,2019-01-28 07:00:00+00:00,2019-01-28 23:47:24+00:00,1,70.000000,4.000000,0.000000,82.250000,4.500000,...,7.0,16.0,Light snow,0.0,86.0,6.0,1023.0,2019-01-29 00:00:00+00:00,2100.055556,0
10288,25117738,R-c172e39f,2019-01-01 07:00:00+00:00,2019-01-02 13:15:00+00:00,0,62.666667,11.000000,0.000000,79.000000,6.000000,...,21.0,7.0,Sunny,0.0,71.0,6.0,1017.0,2019-01-02 13:00:00+00:00,1880.709677,0
10289,11972697,R-b5a32b38,2019-01-19 07:00:00+00:00,2019-01-19 19:37:12+00:00,0,67.500000,9.250000,0.000000,60.000000,3.250000,...,23.0,10.0,Clear,0.0,68.0,6.0,1030.0,2019-01-19 20:00:00+00:00,2297.142857,0
10290,48204624,R-f5d9a32d,2019-01-15 07:00:00+00:00,2019-01-22 03:44:24+00:00,0,56.448276,10.206897,0.010345,61.655172,5.310345,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1757.000000,0


In [36]:
merged_data_weather_traffic.isnull().sum()

truck_id                             0
route_id                             0
departure_date                       0
estimated_arrival                    0
delay                                0
route_avg_temp                      33
route_avg_wind_speed                33
route_avg_precip                    33
route_avg_humidity                  33
route_avg_visibility                33
route_avg_pressure                  33
route_description                   33
estimated_arrival_nearest_hour       0
departure_date_nearest_hour          0
origin_id                          570
destination_id                     570
distance                           570
average_hours                      570
city_id_x                         3957
temp_x                            3957
wind_speed_x                      3957
description_x                     3957
precip_x                          3957
humidity_x                        3957
visibility_x                      3957
pressure_x               

In [41]:
merged_data_weather_traffic = merged_data_weather_traffic.ffill().bfill()


In [42]:
merged_data_weather_traffic.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
route_avg_temp                    0
route_avg_wind_speed              0
route_avg_precip                  0
route_avg_humidity                0
route_avg_visibility              0
route_avg_pressure                0
route_description                 0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
origin_id                         0
destination_id                    0
distance                          0
average_hours                     0
city_id_x                         0
temp_x                            0
wind_speed_x                      0
description_x                     0
precip_x                          0
humidity_x                        0
visibility_x                      0
pressure_x                        0
datetime_x                        0
city_id_y                   

In [43]:
merged_data_weather_traffic

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,...,temp_y,wind_speed_y,description_y,precip_y,humidity_y,visibility_y,pressure_y,datetime_y,avg_no_of_vehicles,accident
0,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:09:00+00:00,1,45.800000,5.800000,0.000000,90.600000,6.000000,...,46.0,4.0,Partly cloudy,0.0,64.0,6.0,1027.0,2019-01-07 16:00:00+00:00,1663.727273,1
1,10234289,R-d053f5bc,2019-01-22 07:00:00+00:00,2019-01-22 22:51:00+00:00,0,70.500000,6.000000,0.000000,78.000000,6.000000,...,46.0,4.0,Partly cloudy,0.0,64.0,6.0,1027.0,2019-01-07 16:00:00+00:00,2269.352941,0
2,26739096,R-bf6ee666,2019-01-07 07:00:00+00:00,2019-01-07 15:54:36+00:00,1,38.666667,12.666667,0.000000,66.333333,3.333333,...,46.0,4.0,Partly cloudy,0.0,64.0,6.0,1027.0,2019-01-07 16:00:00+00:00,1579.500000,0
3,20067958,R-abc9e78b,2019-01-04 07:00:00+00:00,2019-01-05 05:28:12+00:00,0,57.800000,2.200000,0.000000,78.800000,5.600000,...,39.0,6.0,Clear,0.0,78.0,6.0,1018.0,2019-01-05 05:00:00+00:00,1552.391304,0
4,23639934,R-7dac228a,2019-01-31 07:00:00+00:00,2019-01-31 13:40:12+00:00,0,57.666667,6.666667,0.000000,72.666667,6.000000,...,39.0,6.0,Clear,0.0,78.0,6.0,1018.0,2019-01-05 05:00:00+00:00,2259.875000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,30423832,R-f8b24663,2019-01-28 07:00:00+00:00,2019-01-28 23:47:24+00:00,1,70.000000,4.000000,0.000000,82.250000,4.500000,...,7.0,16.0,Light snow,0.0,86.0,6.0,1023.0,2019-01-29 00:00:00+00:00,2100.055556,0
10288,25117738,R-c172e39f,2019-01-01 07:00:00+00:00,2019-01-02 13:15:00+00:00,0,62.666667,11.000000,0.000000,79.000000,6.000000,...,21.0,7.0,Sunny,0.0,71.0,6.0,1017.0,2019-01-02 13:00:00+00:00,1880.709677,0
10289,11972697,R-b5a32b38,2019-01-19 07:00:00+00:00,2019-01-19 19:37:12+00:00,0,67.500000,9.250000,0.000000,60.000000,3.250000,...,23.0,10.0,Clear,0.0,68.0,6.0,1030.0,2019-01-19 20:00:00+00:00,2297.142857,0
10290,48204624,R-f5d9a32d,2019-01-15 07:00:00+00:00,2019-01-22 03:44:24+00:00,0,56.448276,10.206897,0.010345,61.655172,5.310345,...,23.0,10.0,Clear,0.0,68.0,6.0,1030.0,2019-01-19 20:00:00+00:00,1757.000000,0


In [44]:
merged_data_weather_traffic_trucks = pd.merge(merged_data_weather_traffic, trucks_table, on='truck_id', how='left')
merged_data_weather_traffic_trucks

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,...,humidity_y,visibility_y,pressure_y,datetime_y,avg_no_of_vehicles,accident,truck_age,load_capacity_pounds,mileage_mpg,fuel_type
0,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:09:00+00:00,1,45.800000,5.800000,0.000000,90.600000,6.000000,...,64.0,6.0,1027.0,2019-01-07 16:00:00+00:00,1663.727273,1,10,20000.000000,20,diesel
1,10234289,R-d053f5bc,2019-01-22 07:00:00+00:00,2019-01-22 22:51:00+00:00,0,70.500000,6.000000,0.000000,78.000000,6.000000,...,64.0,6.0,1027.0,2019-01-07 16:00:00+00:00,2269.352941,0,8,15000.000000,13,gas
2,26739096,R-bf6ee666,2019-01-07 07:00:00+00:00,2019-01-07 15:54:36+00:00,1,38.666667,12.666667,0.000000,66.333333,3.333333,...,64.0,6.0,1027.0,2019-01-07 16:00:00+00:00,1579.500000,0,8,10000.000000,23,diesel
3,20067958,R-abc9e78b,2019-01-04 07:00:00+00:00,2019-01-05 05:28:12+00:00,0,57.800000,2.200000,0.000000,78.800000,5.600000,...,78.0,6.0,1018.0,2019-01-05 05:00:00+00:00,1552.391304,0,11,3000.000000,19,gas
4,23639934,R-7dac228a,2019-01-31 07:00:00+00:00,2019-01-31 13:40:12+00:00,0,57.666667,6.666667,0.000000,72.666667,6.000000,...,78.0,6.0,1018.0,2019-01-05 05:00:00+00:00,2259.875000,0,9,8610.619469,23,diesel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,30423832,R-f8b24663,2019-01-28 07:00:00+00:00,2019-01-28 23:47:24+00:00,1,70.000000,4.000000,0.000000,82.250000,4.500000,...,86.0,6.0,1023.0,2019-01-29 00:00:00+00:00,2100.055556,0,9,4000.000000,21,diesel
10288,25117738,R-c172e39f,2019-01-01 07:00:00+00:00,2019-01-02 13:15:00+00:00,0,62.666667,11.000000,0.000000,79.000000,6.000000,...,71.0,6.0,1017.0,2019-01-02 13:00:00+00:00,1880.709677,0,9,15000.000000,16,gas
10289,11972697,R-b5a32b38,2019-01-19 07:00:00+00:00,2019-01-19 19:37:12+00:00,0,67.500000,9.250000,0.000000,60.000000,3.250000,...,68.0,6.0,1030.0,2019-01-19 20:00:00+00:00,2297.142857,0,9,3000.000000,18,gas
10290,48204624,R-f5d9a32d,2019-01-15 07:00:00+00:00,2019-01-22 03:44:24+00:00,0,56.448276,10.206897,0.010345,61.655172,5.310345,...,68.0,6.0,1030.0,2019-01-19 20:00:00+00:00,1757.000000,0,11,4000.000000,16,gas


Merge merged_data with truck_data based on 'truck_id' column (Left Join)

In [45]:
final_merge = pd.merge(merged_data_weather_traffic_trucks, drivers_table, left_on='truck_id', right_on = 'vehicle_no', how='left')

Function to check if there is nighttime involved between arrival and departure time

In [46]:
def has_midnight(start, end):
       return int(start.date() != end.date())

final_merge['is_midnight'] = final_merge.apply(lambda row: has_midnight(row['departure_date'], row['estimated_arrival']), axis=1)

In [47]:
final_merge['is_midnight'] = final_merge.apply(lambda row: has_midnight(row['departure_date'], row['estimated_arrival']), axis=1)

In [48]:
final_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:09:00+00:00,1,45.800000,5.800000,0.000000,90.600000,6.000000,...,b0b660a4-5,Nicholas Arroyo,male,50,10,conservative,2,23244180,52.27,1
1,10234289,R-d053f5bc,2019-01-22 07:00:00+00:00,2019-01-22 22:51:00+00:00,0,70.500000,6.000000,0.000000,78.000000,6.000000,...,5050d6b3-b,Maurice Howe,male,53,26,proactive,6,10234289,61.97,0
2,26739096,R-bf6ee666,2019-01-07 07:00:00+00:00,2019-01-07 15:54:36+00:00,1,38.666667,12.666667,0.000000,66.333333,3.333333,...,1c103526-1,Thomas Shields,male,43,13,proactive,4,26739096,63.78,0
3,20067958,R-abc9e78b,2019-01-04 07:00:00+00:00,2019-01-05 05:28:12+00:00,0,57.800000,2.200000,0.000000,78.800000,5.600000,...,0d9c0cc9-8,Dr. Wayne Clark,male,49,10,proactive,7,20067958,60.29,1
4,23639934,R-7dac228a,2019-01-31 07:00:00+00:00,2019-01-31 13:40:12+00:00,0,57.666667,6.666667,0.000000,72.666667,6.000000,...,481191d8-1,Jay Bishop,male,44,9,conservative,7,23639934,42.24,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,30423832,R-f8b24663,2019-01-28 07:00:00+00:00,2019-01-28 23:47:24+00:00,1,70.000000,4.000000,0.000000,82.250000,4.500000,...,ee8a6522-b,Nicholas Kerr,male,53,24,proactive,6,30423832,61.13,0
10288,25117738,R-c172e39f,2019-01-01 07:00:00+00:00,2019-01-02 13:15:00+00:00,0,62.666667,11.000000,0.000000,79.000000,6.000000,...,6961795d-9,Bernard Weber,male,51,22,proactive,7,25117738,62.98,1
10289,11972697,R-b5a32b38,2019-01-19 07:00:00+00:00,2019-01-19 19:37:12+00:00,0,67.500000,9.250000,0.000000,60.000000,3.250000,...,8414372e-3,Robert Archer,male,49,6,conservative,2,11972697,42.61,0
10290,48204624,R-f5d9a32d,2019-01-15 07:00:00+00:00,2019-01-22 03:44:24+00:00,0,56.448276,10.206897,0.010345,61.655172,5.310345,...,1c63864b-0,Timothy Burke,male,44,7,conservative,4,48204624,45.26,1


In [49]:
final_merge = final_merge.drop_duplicates()
final_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:09:00+00:00,1,45.800000,5.800000,0.000000,90.600000,6.000000,...,b0b660a4-5,Nicholas Arroyo,male,50,10,conservative,2,23244180,52.27,1
1,10234289,R-d053f5bc,2019-01-22 07:00:00+00:00,2019-01-22 22:51:00+00:00,0,70.500000,6.000000,0.000000,78.000000,6.000000,...,5050d6b3-b,Maurice Howe,male,53,26,proactive,6,10234289,61.97,0
2,26739096,R-bf6ee666,2019-01-07 07:00:00+00:00,2019-01-07 15:54:36+00:00,1,38.666667,12.666667,0.000000,66.333333,3.333333,...,1c103526-1,Thomas Shields,male,43,13,proactive,4,26739096,63.78,0
3,20067958,R-abc9e78b,2019-01-04 07:00:00+00:00,2019-01-05 05:28:12+00:00,0,57.800000,2.200000,0.000000,78.800000,5.600000,...,0d9c0cc9-8,Dr. Wayne Clark,male,49,10,proactive,7,20067958,60.29,1
4,23639934,R-7dac228a,2019-01-31 07:00:00+00:00,2019-01-31 13:40:12+00:00,0,57.666667,6.666667,0.000000,72.666667,6.000000,...,481191d8-1,Jay Bishop,male,44,9,conservative,7,23639934,42.24,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,30423832,R-f8b24663,2019-01-28 07:00:00+00:00,2019-01-28 23:47:24+00:00,1,70.000000,4.000000,0.000000,82.250000,4.500000,...,ee8a6522-b,Nicholas Kerr,male,53,24,proactive,6,30423832,61.13,0
10288,25117738,R-c172e39f,2019-01-01 07:00:00+00:00,2019-01-02 13:15:00+00:00,0,62.666667,11.000000,0.000000,79.000000,6.000000,...,6961795d-9,Bernard Weber,male,51,22,proactive,7,25117738,62.98,1
10289,11972697,R-b5a32b38,2019-01-19 07:00:00+00:00,2019-01-19 19:37:12+00:00,0,67.500000,9.250000,0.000000,60.000000,3.250000,...,8414372e-3,Robert Archer,male,49,6,conservative,2,11972697,42.61,0
10290,48204624,R-f5d9a32d,2019-01-15 07:00:00+00:00,2019-01-22 03:44:24+00:00,0,56.448276,10.206897,0.010345,61.655172,5.310345,...,1c63864b-0,Timothy Burke,male,44,7,conservative,4,48204624,45.26,1


In [51]:
final_merge = final_merge.drop(columns=['city_id_x','city_id_y'])
final_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,23244180,R-b1e8e9c8,2019-02-06 07:00:00+00:00,2019-02-07 04:09:00+00:00,1,45.800000,5.800000,0.000000,90.600000,6.000000,...,b0b660a4-5,Nicholas Arroyo,male,50,10,conservative,2,23244180,52.27,1
1,10234289,R-d053f5bc,2019-01-22 07:00:00+00:00,2019-01-22 22:51:00+00:00,0,70.500000,6.000000,0.000000,78.000000,6.000000,...,5050d6b3-b,Maurice Howe,male,53,26,proactive,6,10234289,61.97,0
2,26739096,R-bf6ee666,2019-01-07 07:00:00+00:00,2019-01-07 15:54:36+00:00,1,38.666667,12.666667,0.000000,66.333333,3.333333,...,1c103526-1,Thomas Shields,male,43,13,proactive,4,26739096,63.78,0
3,20067958,R-abc9e78b,2019-01-04 07:00:00+00:00,2019-01-05 05:28:12+00:00,0,57.800000,2.200000,0.000000,78.800000,5.600000,...,0d9c0cc9-8,Dr. Wayne Clark,male,49,10,proactive,7,20067958,60.29,1
4,23639934,R-7dac228a,2019-01-31 07:00:00+00:00,2019-01-31 13:40:12+00:00,0,57.666667,6.666667,0.000000,72.666667,6.000000,...,481191d8-1,Jay Bishop,male,44,9,conservative,7,23639934,42.24,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,30423832,R-f8b24663,2019-01-28 07:00:00+00:00,2019-01-28 23:47:24+00:00,1,70.000000,4.000000,0.000000,82.250000,4.500000,...,ee8a6522-b,Nicholas Kerr,male,53,24,proactive,6,30423832,61.13,0
10288,25117738,R-c172e39f,2019-01-01 07:00:00+00:00,2019-01-02 13:15:00+00:00,0,62.666667,11.000000,0.000000,79.000000,6.000000,...,6961795d-9,Bernard Weber,male,51,22,proactive,7,25117738,62.98,1
10289,11972697,R-b5a32b38,2019-01-19 07:00:00+00:00,2019-01-19 19:37:12+00:00,0,67.500000,9.250000,0.000000,60.000000,3.250000,...,8414372e-3,Robert Archer,male,49,6,conservative,2,11972697,42.61,0
10290,48204624,R-f5d9a32d,2019-01-15 07:00:00+00:00,2019-01-22 03:44:24+00:00,0,56.448276,10.206897,0.010345,61.655172,5.310345,...,1c63864b-0,Timothy Burke,male,44,7,conservative,4,48204624,45.26,1
